### Install packages

In [1]:
try: # Install microgrids package in JupyterLite (if run in JupyterLite)
    import piplite
    await piplite.install([ 'ipywidgets', ])
except ImportError:
    pass

### Import packages

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, VBox, interactive_output, Output

### Define fixed variables

In [3]:
# Constants
weightTrain = 200 # Weight of Train in Tons
coeffInertia = 0.1 # Coeffecient of Inertia in %
resistStart = 49 # Starting resistance in N/Ton
motorsNos = 8 # Number of motorised axles
accler = 1.0 # Acceleration value in m/s2

### Function to create the Tractive Effort plot

In [4]:
def plot1(accSpdLmt):
    x = np.linspace(0, 80, num=100)
    
    teStart = weightTrain * resistStart /1000 # Starting Effort in kN
    teRolling = (weightTrain + weightTrain * coeffInertia) * accler # Moving Effort
    teTotal = teStart + teRolling # Total Effort
    teMotor = teTotal / motorsNos # Effort per Motor
    powerMotor = teMotor * accSpdLmt * 1000 / 3600 # Motor Rating

    print(f'Starting Effort:{teStart} kN, Moving Effort:{teRolling}kN')
    print(f'Motor TE:{teMotor}kN, Motor Rating: {powerMotor}')

    te = []
    for i in x:
        if i < accSpdLmt:
            te.append(teMotor)
        else:
            te.append(powerMotor / i * 3600 /1000)
    
    fig, ax = plt.subplots(figsize=(5, 3))  # Set figure size
    ax.plot(x,te)
    ax.set_title(f'Tractive effort is {teMotor}kN at {accSpdLmt}kmph')
    return fig


### Function to create the bar chat for motor power rating

In [5]:
def plot2(accSpdLmt):
    teStart = weightTrain * resistStart /1000 # Starting Effort in kN
    teRolling = (weightTrain + weightTrain * coeffInertia) * accler # Moving Effort
    teTotal = teStart + teRolling # Total Effort
    teMotor = teTotal / motorsNos # Effort per Motor
    powerMotor = teMotor * accSpdLmt * 1000 / 3600 # Motor Rating
    
    fig, ax = plt.subplots(figsize=(5, 3))  # Set figure size
    ax.bar(range(1), powerMotor, width=0.5)
    ax.set_title(f'Motor Power needed is {int(powerMotor)}kW')
    ax.set_ylim(150, 500)
    ax.set_xlim(-5, 5)
    return fig

### Update the paramateric value interactively to observe senstivity

In [6]:
def update(accSpdLmt):
    out1.clear_output(wait=True)
    out2.clear_output(wait=True)
    
    with out1:
        fig1 = plot1(accSpdLmt)
        plt.show(fig1)
    
    with out2:
        fig2 = plot2(accSpdLmt)
        plt.show(fig2)

### Widget and others

In [7]:
# Widgets for interactive plots
accSpdLmt_slider = widgets.FloatSlider(value=30, min=10, max=80, step=0.5, description='Acceleration Limit')

# Outputs for plots
out1 = Output()
out2 = Output()

# Interactive display
ui = HBox([accSpdLmt_slider])
out = VBox([HBox([out1, out2])])

interactive_plot = interactive_output(update, {'accSpdLmt': accSpdLmt_slider, })

### Show interactive plot

In [8]:
display(VBox([ui, out, interactive_plot]))